In [87]:
import torch.utils.data
from Dataloader import *
from Model import *
from Evaluation import *
from Logger import *
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
pca = PCA(n_components=7)

In [88]:

file = pd.read_excel ('SFEW.xlsx')
file = file.fillna(0)
file.columns = ['name', 'target', 'value_1', 'value_2', 'value_3', 'value_4', 'value_5', 'value_6', 'value_7', 'value_8', 'value_9', 'value_10']
file.iloc[:, 2:] = StandardScaler().fit_transform(file.iloc[:, 2:])
accuracies = []
# logger = Logger('./bdnn')

In [91]:
file.iloc[:, 2:9] = pca.fit_transform(file.iloc[:, 2:])

In [92]:
all_losses = []
accuracies = []

In [93]:
# specify the parameters
input_size = 7
hidden_size = 30
embedded_size = 6
num_classes = 7
num_epochs = 200
batch_size = 10
learning_rate = 0.007

for i in range(100):
    
    train_file = file.iloc[:, 1:9].sample(frac=0.8)
    test_file = pd.DataFrame(file.iloc[:, 1:9].merge(train_file, how='outer', indicator=True).loc[lambda x: x['_merge'] == 'left_only']).iloc[:, :-1]

    train_dataset = DataFrameDataset(train_file)
    test_dataset = DataFrameDataset(test_file)


    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


    # construct the network
    net = Net(input_size, hidden_size, num_classes)

    # Loss and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    # criterion1 = nn.MSELoss()
    # criterion2 = nn.CrossEntropyLoss()
    # optimizer = torch.optim.SGD(params=net.parameters(), lr=learning_rate, momentum=0.8)

    # train the model by batch
    # for i in range(10):

    for epoch in range(num_epochs):
        total = 0
        correct = 0
        total_loss = 0
        for step, (batch_x, batch_y) in enumerate(train_loader):
            X = batch_x
            Y = batch_y.long()

            # Forward + Backward + Optimize
            optimizer.zero_grad()  # zero the gradient buffer
            outputs = net(X)
            # print(outputs)
            # loss1 = criterion1(decode, X)
            # loss2 = criterion2(outputs, Y)
            # loss = loss1 + loss2
            loss = criterion(outputs, Y)
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs, 1)
            # calculate and print accuracy
            total = total + predicted.size(0)
            correct = correct + sum(predicted.data.cpu().numpy() == Y.data.cpu().numpy())
            total_loss = total_loss + loss
            accuracy = 100 * correct/total

    # accuracy on testing data
    test_prediction, targets = prediction(net, test_file)
    total = test_prediction.size(0)
    correct = test_prediction.data.numpy() == targets.data.numpy()
    accuracy = 100 * sum(correct)/total
    print('Testing Accuracy: %.2f %%' % accuracy)
    accuracies.append(accuracy)

Testing Accuracy: 25.19 %
Testing Accuracy: 22.22 %
Testing Accuracy: 19.26 %
Testing Accuracy: 22.96 %
Testing Accuracy: 20.74 %
Testing Accuracy: 24.44 %
Testing Accuracy: 19.26 %
Testing Accuracy: 20.00 %
Testing Accuracy: 20.00 %
Testing Accuracy: 20.74 %
Testing Accuracy: 23.70 %
Testing Accuracy: 19.26 %
Testing Accuracy: 23.70 %
Testing Accuracy: 17.78 %
Testing Accuracy: 22.22 %
Testing Accuracy: 21.48 %
Testing Accuracy: 20.74 %
Testing Accuracy: 21.48 %
Testing Accuracy: 23.70 %
Testing Accuracy: 26.67 %
Testing Accuracy: 22.22 %
Testing Accuracy: 24.44 %
Testing Accuracy: 20.00 %
Testing Accuracy: 22.96 %
Testing Accuracy: 25.19 %
Testing Accuracy: 25.93 %
Testing Accuracy: 23.70 %
Testing Accuracy: 22.22 %
Testing Accuracy: 27.41 %
Testing Accuracy: 28.15 %
Testing Accuracy: 23.70 %
Testing Accuracy: 21.48 %
Testing Accuracy: 23.70 %
Testing Accuracy: 22.22 %
Testing Accuracy: 19.26 %
Testing Accuracy: 19.26 %
Testing Accuracy: 22.22 %
Testing Accuracy: 25.93 %
Testing Accu

In [94]:
max(accuracies)

31.85185185185185

In [95]:
sum(accuracies)/len(accuracies)

22.540740740740738

In [96]:
min(accuracies)

15.555555555555555